In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from sklearn import metrics
import cv2
%matplotlib inline

In [ ]:
%run ImageDataLoader.ipynb
%run ImageProcessor.ipynb
%run DataExplorer.ipynb
%run DatasetStatistics.ipynb
%run DuplicateDetector.ipynb
%run OversampledDataset.ipynb
%run BatchVisualizer.ipynb
%run VGG16Model.ipynb
%run ResNet34Model.ipynb
%run EfficientNetModel.ipynb
%run Trainer.ipynb
%run DatasetDivider.ipynb

In [ ]:
TRAIN_DIR = "./new_data/Training"
VAL_DIR = "./new_data/Validation"
TEST_DIR = "./new_data/Testing"

BATCH_SIZE = 64
IMAGE_SIZE = (224, 224)
NUMBER_OF_CLASSES = 4

FINAL_EPOCHS = 50
PATIENCE = 10

In [ ]:
device = (
    torch.accelerator.current_accelerator().type
    if torch.accelerator.is_available()
    else "cpu"
)
print(f"Using {device} device")
if device == "cuda":
    print(f"Accelerator name: {torch.cuda.get_device_name(device)}")
    torch.cuda.empty_cache()

In [ ]:
def set_seed(seed=42):
    random.seed(seed)  # Python random
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (used)
    torch.cuda.manual_seed_all(seed)  # PyTorch all GPUs
    torch.backends.cudnn.deterministic = True  # CUDA deterministic operations
    torch.backends.cudnn.benchmark = False  # Turn off autotuning for reproducity

In [ ]:
all_samples = collect_all_images()

create_directory_structure()
split_data = split_and_copy(all_samples)

print_statistics(split_data)

In [ ]:
loader = ImageDataLoader(TRAIN_DIR, VAL_DIR, TEST_DIR)
all_files = loader.load_all_images()

print(f"Successfully loaded {len(all_files)} images")

In [ ]:
# Policz pliki w oryginalnych folderach
original_train = sum(len(files) for _, _, files in os.walk("./data/Training"))
original_test = sum(len(files) for _, _, files in os.walk("./data/Testing"))
print(
    f"Oryginalne: Training={original_train}, Testing={original_test}, Suma={original_train + original_test}"
)

# Policz pliki w nowych folderach
new_total = sum(len(files) for _, _, files in os.walk("./new_data"))
print(f"Nowe (new_data): {new_total}")

In [ ]:
duplicate_detector = DuplicateDetector(all_files)
duplicate_detector.detect_duplicates()

if len(duplicate_detector.duplicates) > 0:  # If duplicate files are present
    duplicate_detector.remove_duplicates_from_disk()  # Removing duplicates entirely from disk
    all_files = duplicate_detector.get_unique_files()  # Cleaning list with file paths

In [ ]:
loader.print_dataset_class_count()

In [ ]:
processor = ImageProcessor(all_files)

processor.load_grayscale_images()
processor.display_image_grid(batch_size=32, figsize=(18, 9), images_per_row=8)

In [ ]:
explorator = DataExplorer(all_files)
explorator.retrieve_sample_of_images(
    [0, len(all_files) // 2, -1],  # equalize=True
)  # First, middle and last image

explorator.plot_histogram()

In [ ]:
stats = DatasetStatistics(processor.gray_images)

stats.compute_stats()
stats.print_stats()

MEAN, STD = stats.get_normalized_values()

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        # HistogramEqualization(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(12),
        transforms.ColorJitter(
            brightness=0.15, contrast=0.15
        ),  # consider changing to 0.3 to elevate test acc results
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [ ]:
test_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        # HistogramEqualization(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [ ]:
trainset = OversampledDataset(TRAIN_DIR, transform=train_transform)
valset = datasets.ImageFolder(VAL_DIR, transform=test_transform)
testset = datasets.ImageFolder(TEST_DIR, transform=test_transform)

print()
trainset.print_class_distribution()

print()
print(f"Training samples: {len(trainset)}")
print(f"Validation samples: {len(valset)}")
print(f"Test samples: {len(testset)}")
print(f"Classes: {trainset.classes}")

In [ ]:
g = torch.Generator()
g.manual_seed(42)

train_dl = DataLoader(trainset, BATCH_SIZE, shuffle=True, num_workers=0, generator=g)
val_dl = DataLoader(valset, BATCH_SIZE, shuffle=False, num_workers=0)
test_dl = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
vgg16_model = VGG16Model(NUMBER_OF_CLASSES)
optimizer_vgg16 = optim.Adam(vgg16_model.parameters(), lr=0.0001)

trainer_vgg16 = Trainer(vgg16_model, device)
vgg16_history = trainer_vgg16.fit(
    train_dl, val_dl, optimizer_vgg16, FINAL_EPOCHS, PATIENCE
)
vgg16_acc, vgg16_preds, vgg16_labels = trainer_vgg16.test(test_dl)
print(f"\nVGG-16 Test accuracy: {vgg16_acc:.4f}")

In [ ]:
set_seed(42)

resnet34_model = ResNet34Model()
optimizer_resnet = optim.SGD(resnet34_model.parameters(), lr=0.001, momentum=0.9)

trainer_resnet34 = Trainer(resnet34_model, device)
resnet34_history = trainer_resnet34.fit(
    train_dl, val_dl, optimizer_resnet, FINAL_EPOCHS, PATIENCE
)
resnet34_acc, resnet34_preds, resnet34_labels = trainer_resnet34.test(test_dl)
print(f"Test accuracy: {resnet34_acc:.4f}")

In [ ]:
set_seed(42)

efficientnet_model = EfficientNetModel()
optimizer_efficientnet = optim.Adam(efficientnet_model.parameters(), lr=0.001)

trainer_efficientnet = Trainer(efficientnet_model, device)
efficientnet_history = trainer_efficientnet.fit(
    train_dl, val_dl, optimizer_efficientnet, FINAL_EPOCHS, PATIENCE
)
efficientnet_acc, efficientnet_preds, efficientnet_labels = trainer_efficientnet.test(
    test_dl
)
print(f"Test accuracy: {efficientnet_acc:.4f}")

In [ ]:
def plot_train_val_accuracy(train_acc, val_acc, title=None, ax=None):
    if ax is None:
        _, ax = plt.subplots()

    ax.plot(train_acc, "r--", label="Training accuracy")
    ax.plot(val_acc, "b-", label="Validation accuracy")

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    if title is not None:
        ax.set_title(title, fontsize=10)

    ax.grid()
    ax.legend()

In [ ]:
_, axs = plt.subplots(ncols=3, figsize=(15, 5))
plot_train_val_accuracy(
    vgg16_history["train_acc"], vgg16_history["val_acc"], "VGG-16 model", axs[0]
)

plot_train_val_accuracy(
    resnet34_history["train_acc"], resnet34_history["val_acc"], "ResNet34 model", axs[1]
)

plot_train_val_accuracy(
    efficientnet_history["train_acc"],
    efficientnet_history["val_acc"],
    "EfficientNet model",
    axs[2],
)

plt.suptitle("Models training accuracy vs validation accuracy", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
models_config = {
    "VGG-16": VGG16Model,
    "ResNet34": ResNet34Model,
    "EfficientNet_B0": EfficientNetModel,
}

In [ ]:
results = [
    [vgg16_labels, vgg16_preds],
    [resnet34_labels, resnet34_preds],
    [efficientnet_labels, efficientnet_preds],
]

_, axs = plt.subplots(ncols=3, figsize=(28, 8))
i = 0

for model in models_config:
    cm = metrics.confusion_matrix(results[i][0], results[i][1])
    cm_display = metrics.ConfusionMatrixDisplay(cm, display_labels=testset.class_to_idx)
    cm_display.plot(cmap="Blues", ax=axs[i], text_kw={"fontsize": 14})

    axs[i].set_title(f"{model} confusion matrix", y=1.02, fontsize=18)

    axs[i].tick_params(axis="x", rotation=45, labelsize=12)
    axs[i].set_xticklabels(axs[i].get_xticklabels(), ha="right")

    axs[i].tick_params(axis="y", labelsize=12)

    axs[i].set_xlabel("Predicted label", fontsize=14)
    axs[i].set_ylabel("True label", fontsize=14)

    i += 1

plt.suptitle("Confusion matrix for all models", fontsize="22", y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
# TODO: generate new_data once again, test hist eq vs no hist eq, run all experiments once again, add more metrics to rate model's performance, redefine datasetdivider, clean the main files' content